In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_problem_article
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT * FROM (
    SELECT *except(file_creation_ts)
    FROM {catalog_name}.udp_wcm_silver_rop.problem_article
    QUALIFY ROW_NUMBER() OVER (PARTITION BY business_unit, area, sku ORDER BY updated_at DESC) = 1
)
WHERE active_status = 'True';      
""")